<center><img src="images/header.png"></center>

<h1><center>Лекция №6: Введение в анализ социальных сетей</center></h1>
<hr>
<h1><center>Домашняя работа</center></h1>

Данное ДЗ является довольно творческим и оно требует довольно много самостоятельной работы, так что не стоит с ним мешкать =)

Вам предлагается выбрать сообщество (или сообщества) в сети ВКонтакте и проанализировать его (их) контингент. 


**Правила:**
* Отчеты принимаются до **23:59 19 ноября**. Вы должны залить ваш ноутбук в github дать на него ссылку в следующей [форме](https://forms.gle/88d9CfaJN5gPV4bB8).
* Работоспособность будет проверяться через [colab](https://colab.research.google.com) (можете сразу давать ссылку на него). Если ноутбук не будет исполняться (падать по ошибке) то все, что будет написано после ошибки не будет учитываться при формировании оценки. Для самопроверки, выполните аналог restart kernel and run all cells в colab
* Вы должны включить код, с помощью которого вы общались с API вконтакте, но для экономии времени, заранее предусмотреть возможность загрузки уже собранных данных (например, как это было реализовано в ноутбуках по кластеризации через `wget` и dropbox или raw данными в github)

**Пожелания:**
* Выбирайте для анализа небольшие сообщества (до 5к), желательно чтобы вы сами в них состояли - это ускорит вычисления и упростит интерпретацию
* Отнеситесь к этой работе как к исследованию - больше графиков и комментариев!
* Красивые визуализации лучше делать через gephi!

# Загрузка данных (2 балла)

Изучите следующие материалы
* [Знакомство с API](https://vk.com/dev/first_guide)
* [Туториал](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/15-06/lect-vk-api.ipynb) по выгрузке данных с помощью VKAPI
* Про метод execute - [туть](https://vk.com/dev/execute) и [туть](http://www.cyberforum.ru/python-web/thread1730446.html)

Выберите сообщество и загрузите
* Идентификаторы всех [участников сообщества](https://vk.com/dev/groups.getMembers)
* [Списки друзей](https://vk.com/dev/friends.get) участников и их признаки (пол, возраст, университет и все, что посчитаете нужным - нам потом группировать по этим атрибутам)
* [Пересечение](https://vk.com/dev/friends.getMutual) между списками друзей

Таким образом вы должны получить замкнутую группу аккаунтов с их друзьями и связями между ними.

Далее - соберите их этого граф в networkX и сохраните в graphml или gml формате

**Замечания:**
* Вконтакте может ограничить вам скорость запросов, поэтому иногда имеет смысл ставить небольшую паузу (`time.sleep(1)`) между запросами и сохранять предварительные результаты
* Код выгрузки может быть написан как с помощью модуля `vk` так и просто с помощью requests
* Некоторые пользователи могли сделать свой аккаунт приватным, что может вызвать ошибку в методе `getMutual`. Чтобы этого избежать придется проверять каждого на приватность с помощью `users.get` и аттрибута `is_closed`


In [1]:
import vk
import networkx as nx
from PIL import Image     
from urllib.request import urlopen
import pandas as pd
from time import sleep
from tqdm import tqdm

In [2]:
#token = 'Я плохо понимаю, как это работает, но там было написано, что токен обновляется раз в сутки, а еще его никому нельзя показывать.'

In [3]:
#session = vk.Session(access_token = token) 
#api = vk.API(session) 


In [4]:
#group = '185690782'

In [5]:
"""
members = api.groups.getMembers(group_id = group, fields = ['sex', 'bdate', 'city', 'education' ], v = 5.89)
count = members['count']
count
"""

"\nmembers = api.groups.getMembers(group_id = group, fields = ['sex', 'bdate', 'city', 'education' ], v = 5.89)\ncount = members['count']\ncount\n"

In [6]:
#members['items'][5]

In [7]:
"""
i = 0
while(i < count):
    if(members['items'][i].get('deactivated')):
        members['items'].pop(i)
        count-=1
        i-=1
    i+=1
        
members['count'] = count;
"""

"\ni = 0\nwhile(i < count):\n    if(members['items'][i].get('deactivated')):\n        members['items'].pop(i)\n        count-=1\n        i-=1\n    i+=1\n        \nmembers['count'] = count;\n"

In [8]:
"""
for i in tqdm(range(count)):
    if(members['items'][i].get('is_closed')):
        members['items'][i]['friends'] = {'count': 0, 'items': []}
        continue
    friends = api.friends.get(user_id = members['items'][i]['id'], fields = ['sex', 'bdate', 'city', 'education'], v = 5.89)
    if(friends['count'] > 5000):
        print(i, 'count > 5000')
        count = 5000
    if(i % 400 == 0 and i):
        sleep(100)
    sleep(1)
    members['items'][i]['friends'] = friends
"""

"\nfor i in tqdm(range(count)):\n    if(members['items'][i].get('is_closed')):\n        members['items'][i]['friends'] = {'count': 0, 'items': []}\n        continue\n    friends = api.friends.get(user_id = members['items'][i]['id'], fields = ['sex', 'bdate', 'city', 'education'], v = 5.89)\n    if(friends['count'] > 5000):\n        print(i, 'count > 5000')\n        count = 5000\n    if(i % 400 == 0 and i):\n        sleep(100)\n    sleep(1)\n    members['items'][i]['friends'] = friends\n"

In [9]:
"""
for i in range(count):
    j=0
    while(j < members['items'][i]['friends']['count']):
        if(members['items'][i]['friends']['items'][j].get('deactivated')):
            members['items'][i]['friends']['items'].pop(j)
            members['items'][i]['friends']['count']-=1
            j-=1
        j+=1
"""

"\nfor i in range(count):\n    j=0\n    while(j < members['items'][i]['friends']['count']):\n        if(members['items'][i]['friends']['items'][j].get('deactivated')):\n            members['items'][i]['friends']['items'].pop(j)\n            members['items'][i]['friends']['count']-=1\n            j-=1\n        j+=1\n"

In [10]:
### Здесь должен быть код загрузки из VK
### Его можно закомментировать

In [11]:
#g = nx.Graph()

In [12]:
"""
for i in tqdm(range(count)):
    u_id = members['items'][i]['id']
    user = {'first_name' : members['items'][i].get('first_name', '-1'),
            'last_name': members['items'][i].get('last_name', '-1'),
            'sex': members['items'][i].get('sex', '-1'),
            'bdate': members['items'][i].get('bdate', '-1'),
            'city': members['items'][i].get('city', {'title': '-1'}).get('title', '-1'),
            'university_name': members['items'][i].get('university_name', '-1'),
            'faculty_name': members['items'][i].get('faculty_name', '-1')}
    g.add_nodes_from([(u_id, user)])
    
    for j in range(members['items'][i]['friends']['count']):
        friend = members['items'][i]['friends']['items'][j]      
        friend_user =  {'first_name' : friend.get('first_name', '-1'),
                        'last_name': friend.get('last_name', '-1'),
                        'sex': friend.get('sex', '-1'),
                        'bdate': friend.get('bdate', '-1'),
                        'city': friend.get('city', {'title': '-1'}).get('title', '-1'),
                        'university_name': friend.get('university_name', '-1'),
                        'faculty_name': friend.get('faculty_name', '-1')}
        g.add_nodes_from([(friend['id'], friend_user)])
        g.add_edge( u_id, friend['id'])   
"""

"\nfor i in tqdm(range(count)):\n    u_id = members['items'][i]['id']\n    user = {'first_name' : members['items'][i].get('first_name', '-1'),\n            'last_name': members['items'][i].get('last_name', '-1'),\n            'sex': members['items'][i].get('sex', '-1'),\n            'bdate': members['items'][i].get('bdate', '-1'),\n            'city': members['items'][i].get('city', {'title': '-1'}).get('title', '-1'),\n            'university_name': members['items'][i].get('university_name', '-1'),\n            'faculty_name': members['items'][i].get('faculty_name', '-1')}\n    g.add_nodes_from([(u_id, user)])\n    \n    for j in range(members['items'][i]['friends']['count']):\n        friend = members['items'][i]['friends']['items'][j]      \n        friend_user =  {'first_name' : friend.get('first_name', '-1'),\n                        'last_name': friend.get('last_name', '-1'),\n                        'sex': friend.get('sex', '-1'),\n                        'bdate': friend.get('bd

In [13]:
#len(g.nodes)

In [14]:
#nx.write_graphml(g, 'my_graph_208_group.graphml')

In [15]:
### Здесь должен быть код преобразования в граф и сохранение в файл
### Его тоже можно закомментировать

In [16]:
import wget 
wget.download("https://raw.githubusercontent.com/Sanechka978/networks/main/my_graph_208_group.graphml", "my_graph_208.graphml")

'my_graph_208 (1).graphml'

In [17]:
graph = nx.read_graphml('my_graph_208.graphml')

In [18]:
len(graph.nodes)

4885

In [19]:
### Здесь код, который загружает сохраненную сеть и все необходимые данные через wget

# Визуализация (1 балл)

* Откройте полученный граф в gephi
* Выберите наиболее удачный на ваш взгляд алгоритм прорисовки
* Добавьте изменение размера вершины в зависимости от степени вершины
* Вставьте скрин-шот ниже

In [20]:
### Ваши картинки и комментарии ниже

<img src='./graph' width="1000"/>

In [21]:
#По университетам

<img src='./graph_univers' width="1000"/>

# Центральности вершин (2 балла)

Возможно, перед тем как пойти дальше, стоит немного почитстить граф - с помощью [kcore](https://www.google.com/search?client=firefox-b-d&q=networkx+kcore)

* Найдите наиболее влиятельные вершины с точки зрения degree, closeness и betweenness centrality
* Выделите их на картинке в gephi

In [22]:
### Ваш код, картинки и комментарии ниже
import operator

In [23]:
degr = nx.centrality.degree_centrality(graph)

sorted_degr = sorted(degr.items(), key=operator.itemgetter(1), reverse=True)
degr_cent = {k: v for k, v in sorted_degr}

degr_cent

{'14480835': 0.13431613431613432,
 '250556735': 0.10892710892710894,
 '164896348': 0.10339885339885341,
 '100929764': 0.07637182637182638,
 '60206644': 0.07166257166257167,
 '170751181': 0.05630630630630631,
 '179563115': 0.04893529893529894,
 '153690762': 0.04893529893529894,
 '196339166': 0.04852579852579853,
 '237352270': 0.04504504504504505,
 '233562019': 0.041154791154791155,
 '225719975': 0.03726453726453727,
 '138594254': 0.03562653562653563,
 '209936993': 0.034602784602784604,
 '13702663': 0.0341932841932842,
 '103670132': 0.03357903357903358,
 '55532434': 0.032555282555282554,
 '138623510': 0.029279279279279282,
 '168106692': 0.02620802620802621,
 '289841871': 0.024365274365274366,
 '333215195': 0.024160524160524163,
 '263286492': 0.01760851760851761,
 '180033124': 0.016175266175266177,
 '442333588': 0.0128992628992629,
 '385509098': 0.010237510237510239,
 '163661178': 0.010237510237510239,
 '490363859': 0.007371007371007371,
 '505550202': 0.005323505323505324,
 '354226038': 0

In [24]:
closeness = nx.centrality.closeness_centrality(graph)
sorted_closeness = sorted(closeness.items(), key=operator.itemgetter(1), reverse=True)
closeness_cent = {k: v for k, v in sorted_closeness}

closeness_cent

{'250556735': 0.4731182795698925,
 '164896348': 0.4670555608683179,
 '138623510': 0.46223736513344693,
 '263286492': 0.4604072398190045,
 '179563115': 0.4560224089635854,
 '354226038': 0.45500279485746226,
 '237352270': 0.45121951219512196,
 '100929764': 0.4446062812926718,
 '168106692': 0.4308017994178354,
 '55532434': 0.4200206398348813,
 '584293094': 0.408395350781838,
 '385509098': 0.406187624750499,
 '153690762': 0.4060863058119232,
 '14480835': 0.406052544063851,
 '138594254': 0.40460608068925524,
 '233562019': 0.4036697247706422,
 '289841871': 0.40171080769863465,
 '333215195': 0.39875898105813196,
 '170751181': 0.3963963963963964,
 '209936993': 0.39610705596107054,
 '163661178': 0.39399806389157793,
 '566590946': 0.39090763566511927,
 '225719975': 0.3903764687075374,
 '13702663': 0.39012700694943686,
 '210203981': 0.3899712551900351,
 '111945586': 0.38740382327278494,
 '3322376': 0.3851127582400252,
 '196339166': 0.38067030397505847,
 '490363859': 0.3762422001386642,
 '18003312

In [ ]:
betw = nx.betweenness_centrality(graph) 
sorted_betw = sorted(betw.items(), key=operator.itemgetter(1), reverse=True)
betw_cent = {k: v for k, v in sorted_betw}

betw_cent

# Сообщества... в сообществе (2 балла)

В этом задании надо примерить 2 алгоритма выявления сообществ и сравнить их результаты

Выберите алгоритм выделения сообществ на графах, например
* label propagation или 
* алгоритм на основе edge betweeness

и алгоритм на основе похожести вершин, например 
* расчитатайте попарную похожесть по jaccard, косинусному расстоянию или корреляцию
* при необходимости переведите "похожесть" в "расстояние" (чем больше значение тем "дальше" объекты друг от друга)
* подайте матрицу попарных расстояний в алгоритм кластеризации, например в DBSCAN

Для обоих вариантов
* Выделите сообщества, опираясь на максимальное значение модулярности для полученного разбиения
* Изобразите граф с вершинами, раскрашенными в цвет своего сообщества в gephi
* Попробуйте проинтерпретировать полученные сообщества, кто эти люди, почему они связаны?
* Какой метод на ваш взгляд сработал лучше?

In [ ]:
### Ваш код, картинки и комментарии ниже